In [1]:
# helper ipynb file for targeted supervised ocntrastive learning

In [2]:
# Check what happens with the reserved vectors concept

# Fixed Simplex creation and angle printing

In [356]:
# Load base_prototypes for the model
model_dir = "/BS/fscil/work/code/CEC-CVPR2021/checkpoint/mini_imagenet/base_supcon_sr/test_2_ft_dot-ft_dot-data_init-start_1/Epo_100-Lr_0.1000-Step_40-Gam_0.10-Bs_128-Mom_0.90-ftLR_0.100-ftnEpoch_60-jEpoch_100-opt_sgd-vm_hm-ls_0.00-ma_0.30/session0_max_acc.pth"
best_model_dict = torch.load(model_dir)["params"]

In [361]:
base_prototypes = best_model_dict["module.fc.base_fc.weight"]

In [364]:
compute_angles(normalize(base_prototypes))

(89.770515, 85.42505)

In [375]:
radius = 1.0
temperature = 0.5

def normalize(x): 
    x = x / torch.norm(x, dim=1, p=2, keepdim=True)
    x = x*radius
    return x

def compute_angles(vectors):
    # Compute angles for each vector with all others from 
    # Compute angles with the closesnt neighbour only
    proto = vectors.cpu().numpy()
    dot = np.matmul(proto, proto.T)
    dot = dot.clip(min=0, max=1)
    theta = np.arccos(dot)
    np.fill_diagonal(theta, np.nan)
    theta = theta[~np.isnan(theta)].reshape(theta.shape[0], theta.shape[1] - 1)
    
    # Compute the mean for the angle to the nearest neighbour
    avg_angle_close = theta.min(axis = 1).mean()
    
    avg_angle = theta.mean()
#     print("Average angle with all other prototypes (deg): ", np.rad2deg(avg_angle))
    
    # Average angle to the nearest neighbour

    return np.rad2deg(avg_angle), np.rad2deg(avg_angle_close)


foo_points = base_prototypes
foo_points = normalize(foo_points)

points = foo_points.clone()
points = normalize(points)
points = torch.nn.Parameter(points)
opt = torch.optim.SGD([points], lr=5)

tqdm_gen = tqdm(range(20000))
best_angle = 0
for idx, _ in enumerate(tqdm_gen):
    # # Combining prototypes but only optimising the reserve vector
    comb = torch.cat((points, foo_points), axis = 0)
    
    # Compute the cosine similarity.
#     sim = F.cosine_similarity(points[None,:,:], points[:,None,:], dim=-1)
#     l = torch.log(torch.exp(sim/temperature).sum(axis = 1)).sum() / points.shape[0]
    sim = F.cosine_similarity(comb[None,:,:], comb[:,None,:], dim=-1)
    l = torch.log(torch.exp(sim/temperature).sum(axis = 1)).sum() / comb.shape[0]

    opt.zero_grad()
    l.backward()
    opt.step()
    points.data = normalize(points.data)

    # TODO: Print average angle between all vectors
    # Print all angles vs just base angles vs just reserve angles
    # Print angles with closesnt neighbour
    curr_angle, curr_angle_close = compute_angles(torch.cat((points, foo_points), axis = 0).detach())
    if curr_angle > best_angle: # best angle is the angle with which the separation is maximised
        best_angle = curr_angle
    
    curr_angle_base, curr_angle_base_close = compute_angles(foo_points)
#     curr_angle_novel, curr_angle_novel_close = compute_angles(points)
    
    tqdm_gen.set_description(f"Loss = {l:.5f}, Best Avg Angle [all]: {best_angle:.3f}, Average Angle rv+base [close]: {curr_angle_close:.3f}, Avg Angle Base [close]: {curr_angle_base_close:.3f}")



# Setting Reserved vectors
# self.rv = points.data
# self.register_buffer('rv', points.data)

Loss = 4.82395, Best Avg Angle [all]: 89.934, Average Angle rv+base [close]: 87.666, Avg Angle Base [close]: 85.425: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:48<00:00, 415.81it/s]


In [286]:
compute_angles(points.data)

Average angle with all other prototypes (deg):  89.51694


In [3]:
# Make cub random

In [9]:
x = """CUB_200_2011/images/101.White_Pelican/White_Pelican_0081_96148.jpg
CUB_200_2011/images/101.White_Pelican/White_Pelican_0075_96422.jpg
CUB_200_2011/images/101.White_Pelican/White_Pelican_0026_95832.jpg
CUB_200_2011/images/101.White_Pelican/White_Pelican_0022_95897.jpg
CUB_200_2011/images/101.White_Pelican/White_Pelican_0044_96028.jpg
CUB_200_2011/images/102.Western_Wood_Pewee/Western_Wood_Pewee_0072_98035.jpg
CUB_200_2011/images/102.Western_Wood_Pewee/Western_Wood_Pewee_0004_98257.jpg
CUB_200_2011/images/102.Western_Wood_Pewee/Western_Wood_Pewee_0060_795045.jpg
CUB_200_2011/images/102.Western_Wood_Pewee/Western_Wood_Pewee_0039_795063.jpg
CUB_200_2011/images/102.Western_Wood_Pewee/Western_Wood_Pewee_0040_795051.jpg
CUB_200_2011/images/103.Sayornis/Sayornis_0099_98593.jpg
CUB_200_2011/images/103.Sayornis/Sayornis_0133_99129.jpg
CUB_200_2011/images/103.Sayornis/Sayornis_0098_98419.jpg
CUB_200_2011/images/103.Sayornis/Sayornis_0011_98610.jpg
CUB_200_2011/images/103.Sayornis/Sayornis_0114_98976.jpg
CUB_200_2011/images/104.American_Pipit/American_Pipit_0037_99954.jpg
CUB_200_2011/images/104.American_Pipit/American_Pipit_0067_100237.jpg
CUB_200_2011/images/104.American_Pipit/American_Pipit_0019_99810.jpg
CUB_200_2011/images/104.American_Pipit/American_Pipit_0058_100218.jpg
CUB_200_2011/images/104.American_Pipit/American_Pipit_0113_99939.jpg
CUB_200_2011/images/105.Whip_poor_Will/Whip_Poor_Will_0038_100443.jpg
CUB_200_2011/images/105.Whip_poor_Will/Whip_Poor_Will_0018_796403.jpg
CUB_200_2011/images/105.Whip_poor_Will/Whip_Poor_Will_0013_796439.jpg
CUB_200_2011/images/105.Whip_poor_Will/Whip_Poor_Will_0026_100456.jpg
CUB_200_2011/images/105.Whip_poor_Will/Whip_Poor_Will_0004_100479.jpg
CUB_200_2011/images/106.Horned_Puffin/Horned_Puffin_0004_100733.jpg
CUB_200_2011/images/106.Horned_Puffin/Horned_Puffin_0028_100765.jpg
CUB_200_2011/images/106.Horned_Puffin/Horned_Puffin_0062_100693.jpg
CUB_200_2011/images/106.Horned_Puffin/Horned_Puffin_0042_100760.jpg
CUB_200_2011/images/106.Horned_Puffin/Horned_Puffin_0030_100725.jpg
CUB_200_2011/images/107.Common_Raven/Common_Raven_0009_102112.jpg
CUB_200_2011/images/107.Common_Raven/Common_Raven_0068_101216.jpg
CUB_200_2011/images/107.Common_Raven/Common_Raven_0099_102534.jpg
CUB_200_2011/images/107.Common_Raven/Common_Raven_0001_101213.jpg
CUB_200_2011/images/107.Common_Raven/Common_Raven_0095_101831.jpg
CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0063_797361.jpg
CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0050_797374.jpg
CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0010_797350.jpg
CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0002_797370.jpg
CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0026_797357.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0036_103231.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0071_103266.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0085_103155.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0056_103241.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0049_103176.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0106_104216.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0086_104755.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0124_104141.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0117_104227.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0036_104173.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0127_105742.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0018_26407.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0019_106132.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0011_104921.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0033_105686.jpg
CUB_200_2011/images/112.Great_Grey_Shrike/Great_Grey_Shrike_0092_797048.jpg
CUB_200_2011/images/112.Great_Grey_Shrike/Great_Grey_Shrike_0042_797056.jpg
CUB_200_2011/images/112.Great_Grey_Shrike/Great_Grey_Shrike_0049_797025.jpg
CUB_200_2011/images/112.Great_Grey_Shrike/Great_Grey_Shrike_0083_797051.jpg
CUB_200_2011/images/112.Great_Grey_Shrike/Great_Grey_Shrike_0063_797042.jpg
CUB_200_2011/images/113.Baird_Sparrow/Baird_Sparrow_0021_794576.jpg
CUB_200_2011/images/113.Baird_Sparrow/Baird_Sparrow_0018_794584.jpg
CUB_200_2011/images/113.Baird_Sparrow/Baird_Sparrow_0025_794564.jpg
CUB_200_2011/images/113.Baird_Sparrow/Baird_Sparrow_0041_794582.jpg
CUB_200_2011/images/113.Baird_Sparrow/Baird_Sparrow_0036_794572.jpg
CUB_200_2011/images/114.Black_throated_Sparrow/Black_Throated_Sparrow_0019_107192.jpg
CUB_200_2011/images/114.Black_throated_Sparrow/Black_Throated_Sparrow_0088_107220.jpg
CUB_200_2011/images/114.Black_throated_Sparrow/Black_Throated_Sparrow_0097_106935.jpg
CUB_200_2011/images/114.Black_throated_Sparrow/Black_Throated_Sparrow_0055_107213.jpg
CUB_200_2011/images/114.Black_throated_Sparrow/Black_Throated_Sparrow_0010_107375.jpg
CUB_200_2011/images/115.Brewer_Sparrow/Brewer_Sparrow_0068_107422.jpg
CUB_200_2011/images/115.Brewer_Sparrow/Brewer_Sparrow_0036_107451.jpg
CUB_200_2011/images/115.Brewer_Sparrow/Brewer_Sparrow_0041_796711.jpg
CUB_200_2011/images/115.Brewer_Sparrow/Brewer_Sparrow_0014_107435.jpg
CUB_200_2011/images/115.Brewer_Sparrow/Brewer_Sparrow_0076_107393.jpg
CUB_200_2011/images/116.Chipping_Sparrow/Chipping_Sparrow_0064_108204.jpg
CUB_200_2011/images/116.Chipping_Sparrow/Chipping_Sparrow_0038_109234.jpg
CUB_200_2011/images/116.Chipping_Sparrow/Chipping_Sparrow_0098_108644.jpg
CUB_200_2011/images/116.Chipping_Sparrow/Chipping_Sparrow_0110_108974.jpg
CUB_200_2011/images/116.Chipping_Sparrow/Chipping_Sparrow_0023_108684.jpg
CUB_200_2011/images/117.Clay_colored_Sparrow/Clay_Colored_Sparrow_0104_110699.jpg
CUB_200_2011/images/117.Clay_colored_Sparrow/Clay_Colored_Sparrow_0098_110735.jpg
CUB_200_2011/images/117.Clay_colored_Sparrow/Clay_Colored_Sparrow_0003_110672.jpg
CUB_200_2011/images/117.Clay_colored_Sparrow/Clay_Colored_Sparrow_0029_110720.jpg
CUB_200_2011/images/117.Clay_colored_Sparrow/Clay_Colored_Sparrow_0087_110946.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0092_111413.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0111_112968.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0080_111099.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0130_110985.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0053_111388.jpg
CUB_200_2011/images/119.Field_Sparrow/Field_Sparrow_0069_113827.jpg
CUB_200_2011/images/119.Field_Sparrow/Field_Sparrow_0130_113846.jpg
CUB_200_2011/images/119.Field_Sparrow/Field_Sparrow_0091_113486.jpg
CUB_200_2011/images/119.Field_Sparrow/Field_Sparrow_0043_113607.jpg
CUB_200_2011/images/119.Field_Sparrow/Field_Sparrow_0108_114154.jpg
CUB_200_2011/images/120.Fox_Sparrow/Fox_Sparrow_0104_114908.jpg
CUB_200_2011/images/120.Fox_Sparrow/Fox_Sparrow_0086_115484.jpg
CUB_200_2011/images/120.Fox_Sparrow/Fox_Sparrow_0055_114809.jpg
CUB_200_2011/images/120.Fox_Sparrow/Fox_Sparrow_0012_115324.jpg
CUB_200_2011/images/120.Fox_Sparrow/Fox_Sparrow_0035_114866.jpg
CUB_200_2011/images/121.Grasshopper_Sparrow/Grasshopper_Sparrow_0014_116129.jpg
CUB_200_2011/images/121.Grasshopper_Sparrow/Grasshopper_Sparrow_0114_116160.jpg
CUB_200_2011/images/121.Grasshopper_Sparrow/Grasshopper_Sparrow_0068_115799.jpg
CUB_200_2011/images/121.Grasshopper_Sparrow/Grasshopper_Sparrow_0110_115644.jpg
CUB_200_2011/images/121.Grasshopper_Sparrow/Grasshopper_Sparrow_0042_115638.jpg
CUB_200_2011/images/122.Harris_Sparrow/Harris_Sparrow_0006_116364.jpg
CUB_200_2011/images/122.Harris_Sparrow/Harris_Sparrow_0018_116402.jpg
CUB_200_2011/images/122.Harris_Sparrow/Harris_Sparrow_0026_116620.jpg
CUB_200_2011/images/122.Harris_Sparrow/Harris_Sparrow_0020_116379.jpg
CUB_200_2011/images/122.Harris_Sparrow/Harris_Sparrow_0011_116597.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0023_796582.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0052_796599.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0054_116850.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0064_796573.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0070_796571.jpg
CUB_200_2011/images/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0040_117088.jpg
CUB_200_2011/images/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0072_795230.jpg
CUB_200_2011/images/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0068_795180.jpg
CUB_200_2011/images/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0081_795215.jpg
CUB_200_2011/images/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0032_795186.jpg
CUB_200_2011/images/125.Lincoln_Sparrow/Lincoln_Sparrow_0084_117492.jpg
CUB_200_2011/images/125.Lincoln_Sparrow/Lincoln_Sparrow_0009_117535.jpg
CUB_200_2011/images/125.Lincoln_Sparrow/Lincoln_Sparrow_0014_117883.jpg
CUB_200_2011/images/125.Lincoln_Sparrow/Lincoln_Sparrow_0042_117507.jpg
CUB_200_2011/images/125.Lincoln_Sparrow/Lincoln_Sparrow_0072_117951.jpg
CUB_200_2011/images/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0056_117974.jpg
CUB_200_2011/images/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0002_796908.jpg
CUB_200_2011/images/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0051_796902.jpg
CUB_200_2011/images/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0014_796906.jpg
CUB_200_2011/images/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0077_796913.jpg
CUB_200_2011/images/127.Savannah_Sparrow/Savannah_Sparrow_0049_119596.jpg
CUB_200_2011/images/127.Savannah_Sparrow/Savannah_Sparrow_0118_118603.jpg
CUB_200_2011/images/127.Savannah_Sparrow/Savannah_Sparrow_0068_119972.jpg
CUB_200_2011/images/127.Savannah_Sparrow/Savannah_Sparrow_0052_118583.jpg
CUB_200_2011/images/127.Savannah_Sparrow/Savannah_Sparrow_0054_120057.jpg
CUB_200_2011/images/128.Seaside_Sparrow/Seaside_Sparrow_0001_120720.jpg
CUB_200_2011/images/128.Seaside_Sparrow/Seaside_Sparrow_0048_120758.jpg
CUB_200_2011/images/128.Seaside_Sparrow/Seaside_Sparrow_0042_796528.jpg
CUB_200_2011/images/128.Seaside_Sparrow/Seaside_Sparrow_0049_120735.jpg
CUB_200_2011/images/128.Seaside_Sparrow/Seaside_Sparrow_0035_796533.jpg
CUB_200_2011/images/129.Song_Sparrow/Song_Sparrow_0046_121903.jpg
CUB_200_2011/images/129.Song_Sparrow/Song_Sparrow_0055_121158.jpg
CUB_200_2011/images/129.Song_Sparrow/Song_Sparrow_0107_120990.jpg
CUB_200_2011/images/129.Song_Sparrow/Song_Sparrow_0091_121651.jpg
CUB_200_2011/images/129.Song_Sparrow/Song_Sparrow_0087_121062.jpg
CUB_200_2011/images/130.Tree_Sparrow/Tree_Sparrow_0094_124974.jpg
CUB_200_2011/images/130.Tree_Sparrow/Tree_Sparrow_0123_125324.jpg
CUB_200_2011/images/130.Tree_Sparrow/Tree_Sparrow_0041_123497.jpg
CUB_200_2011/images/130.Tree_Sparrow/Tree_Sparrow_0086_123751.jpg
CUB_200_2011/images/130.Tree_Sparrow/Tree_Sparrow_0119_124114.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0079_125579.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0080_125606.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0084_125532.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0094_125602.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0019_125558.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0068_126156.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0100_126267.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0072_127080.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0033_127728.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0095_127118.jpg
CUB_200_2011/images/133.White_throated_Sparrow/White_Throated_Sparrow_0125_128832.jpg
CUB_200_2011/images/133.White_throated_Sparrow/White_Throated_Sparrow_0056_128906.jpg
CUB_200_2011/images/133.White_throated_Sparrow/White_Throated_Sparrow_0085_129180.jpg
CUB_200_2011/images/133.White_throated_Sparrow/White_Throated_Sparrow_0042_128899.jpg
CUB_200_2011/images/133.White_throated_Sparrow/White_Throated_Sparrow_0021_128804.jpg
CUB_200_2011/images/134.Cape_Glossy_Starling/Cape_Glossy_Starling_0096_129388.jpg
CUB_200_2011/images/134.Cape_Glossy_Starling/Cape_Glossy_Starling_0046_129434.jpg
CUB_200_2011/images/134.Cape_Glossy_Starling/Cape_Glossy_Starling_0043_129358.jpg
CUB_200_2011/images/134.Cape_Glossy_Starling/Cape_Glossy_Starling_0019_129407.jpg
CUB_200_2011/images/134.Cape_Glossy_Starling/Cape_Glossy_Starling_0067_129380.jpg
CUB_200_2011/images/135.Bank_Swallow/Bank_Swallow_0003_129623.jpg
CUB_200_2011/images/135.Bank_Swallow/Bank_Swallow_0045_129483.jpg
CUB_200_2011/images/135.Bank_Swallow/Bank_Swallow_0020_129747.jpg
CUB_200_2011/images/135.Bank_Swallow/Bank_Swallow_0067_129959.jpg
CUB_200_2011/images/135.Bank_Swallow/Bank_Swallow_0053_129501.jpg
CUB_200_2011/images/136.Barn_Swallow/Barn_Swallow_0018_130709.jpg
CUB_200_2011/images/136.Barn_Swallow/Barn_Swallow_0048_132793.jpg
CUB_200_2011/images/136.Barn_Swallow/Barn_Swallow_0070_130127.jpg
CUB_200_2011/images/136.Barn_Swallow/Barn_Swallow_0066_130214.jpg
CUB_200_2011/images/136.Barn_Swallow/Barn_Swallow_0049_130181.jpg
CUB_200_2011/images/137.Cliff_Swallow/Cliff_Swallow_0018_132974.jpg
CUB_200_2011/images/137.Cliff_Swallow/Cliff_Swallow_0023_134314.jpg
CUB_200_2011/images/137.Cliff_Swallow/Cliff_Swallow_0066_133206.jpg
CUB_200_2011/images/137.Cliff_Swallow/Cliff_Swallow_0050_134054.jpg
CUB_200_2011/images/137.Cliff_Swallow/Cliff_Swallow_0075_134516.jpg
CUB_200_2011/images/138.Tree_Swallow/Tree_Swallow_0087_137354.jpg
CUB_200_2011/images/138.Tree_Swallow/Tree_Swallow_0043_136878.jpg
CUB_200_2011/images/138.Tree_Swallow/Tree_Swallow_0111_135253.jpg
CUB_200_2011/images/138.Tree_Swallow/Tree_Swallow_0108_135068.jpg
CUB_200_2011/images/138.Tree_Swallow/Tree_Swallow_0064_136322.jpg
CUB_200_2011/images/139.Scarlet_Tanager/Scarlet_Tanager_0107_138577.jpg
CUB_200_2011/images/139.Scarlet_Tanager/Scarlet_Tanager_0077_137626.jpg
CUB_200_2011/images/139.Scarlet_Tanager/Scarlet_Tanager_0040_137885.jpg
CUB_200_2011/images/139.Scarlet_Tanager/Scarlet_Tanager_0033_137603.jpg
CUB_200_2011/images/139.Scarlet_Tanager/Scarlet_Tanager_0132_138001.jpg
CUB_200_2011/images/140.Summer_Tanager/Summer_Tanager_0032_140425.jpg
CUB_200_2011/images/140.Summer_Tanager/Summer_Tanager_0046_139802.jpg
CUB_200_2011/images/140.Summer_Tanager/Summer_Tanager_0111_139605.jpg
CUB_200_2011/images/140.Summer_Tanager/Summer_Tanager_0116_139923.jpg
CUB_200_2011/images/140.Summer_Tanager/Summer_Tanager_0095_139882.jpg
CUB_200_2011/images/141.Artic_Tern/Artic_Tern_0055_141524.jpg
CUB_200_2011/images/141.Artic_Tern/Artic_Tern_0124_142121.jpg
CUB_200_2011/images/141.Artic_Tern/Artic_Tern_0133_141069.jpg
CUB_200_2011/images/141.Artic_Tern/Artic_Tern_0111_143101.jpg
CUB_200_2011/images/141.Artic_Tern/Artic_Tern_0107_141181.jpg
CUB_200_2011/images/142.Black_Tern/Black_Tern_0079_143998.jpg
CUB_200_2011/images/142.Black_Tern/Black_Tern_0082_144372.jpg
CUB_200_2011/images/142.Black_Tern/Black_Tern_0029_144140.jpg
CUB_200_2011/images/142.Black_Tern/Black_Tern_0066_144541.jpg
CUB_200_2011/images/142.Black_Tern/Black_Tern_0046_144229.jpg
CUB_200_2011/images/143.Caspian_Tern/Caspian_Tern_0009_145057.jpg
CUB_200_2011/images/143.Caspian_Tern/Caspian_Tern_0116_145607.jpg
CUB_200_2011/images/143.Caspian_Tern/Caspian_Tern_0123_145774.jpg
CUB_200_2011/images/143.Caspian_Tern/Caspian_Tern_0006_145594.jpg
CUB_200_2011/images/143.Caspian_Tern/Caspian_Tern_0013_145553.jpg
CUB_200_2011/images/144.Common_Tern/Common_Tern_0071_148796.jpg
CUB_200_2011/images/144.Common_Tern/Common_Tern_0077_149196.jpg
CUB_200_2011/images/144.Common_Tern/Common_Tern_0030_147825.jpg
CUB_200_2011/images/144.Common_Tern/Common_Tern_0095_149960.jpg
CUB_200_2011/images/144.Common_Tern/Common_Tern_0083_148096.jpg
CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0009_150954.jpg
CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0045_150752.jpg
CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0046_150905.jpg
CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0103_150493.jpg
CUB_200_2011/images/145.Elegant_Tern/Elegant_Tern_0004_150948.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0027_151456.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0077_152255.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0125_151399.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0045_151227.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0119_152709.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0092_153361.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0020_153458.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0060_153190.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0119_153950.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0037_153637.jpg
CUB_200_2011/images/148.Green_tailed_Towhee/Green_Tailed_Towhee_0018_154825.jpg
CUB_200_2011/images/148.Green_tailed_Towhee/Green_Tailed_Towhee_0070_154844.jpg
CUB_200_2011/images/148.Green_tailed_Towhee/Green_Tailed_Towhee_0064_154771.jpg
CUB_200_2011/images/148.Green_tailed_Towhee/Green_Tailed_Towhee_0058_797399.jpg
CUB_200_2011/images/148.Green_tailed_Towhee/Green_Tailed_Towhee_0060_154820.jpg
CUB_200_2011/images/149.Brown_Thrasher/Brown_Thrasher_0013_155329.jpg
CUB_200_2011/images/149.Brown_Thrasher/Brown_Thrasher_0079_155394.jpg
CUB_200_2011/images/149.Brown_Thrasher/Brown_Thrasher_0019_155216.jpg
CUB_200_2011/images/149.Brown_Thrasher/Brown_Thrasher_0051_155344.jpg
CUB_200_2011/images/149.Brown_Thrasher/Brown_Thrasher_0081_155256.jpg
CUB_200_2011/images/150.Sage_Thrasher/Sage_Thrasher_0033_155511.jpg
CUB_200_2011/images/150.Sage_Thrasher/Sage_Thrasher_0069_155544.jpg
CUB_200_2011/images/150.Sage_Thrasher/Sage_Thrasher_0096_155449.jpg
CUB_200_2011/images/150.Sage_Thrasher/Sage_Thrasher_0104_155529.jpg
CUB_200_2011/images/150.Sage_Thrasher/Sage_Thrasher_0070_155732.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0012_797473.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0007_797481.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0020_797461.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0053_797478.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0003_797467.jpg
CUB_200_2011/images/152.Blue_headed_Vireo/Blue_Headed_Vireo_0097_156272.jpg
CUB_200_2011/images/152.Blue_headed_Vireo/Blue_Headed_Vireo_0019_156311.jpg
CUB_200_2011/images/152.Blue_headed_Vireo/Blue_Headed_Vireo_0121_156233.jpg
CUB_200_2011/images/152.Blue_headed_Vireo/Blue_Headed_Vireo_0011_156276.jpg
CUB_200_2011/images/152.Blue_headed_Vireo/Blue_Headed_Vireo_0119_156259.jpg
CUB_200_2011/images/153.Philadelphia_Vireo/Philadelphia_Vireo_0078_794776.jpg
CUB_200_2011/images/153.Philadelphia_Vireo/Philadelphia_Vireo_0039_794794.jpg
CUB_200_2011/images/153.Philadelphia_Vireo/Philadelphia_Vireo_0068_794763.jpg
CUB_200_2011/images/153.Philadelphia_Vireo/Philadelphia_Vireo_0012_794785.jpg
CUB_200_2011/images/153.Philadelphia_Vireo/Philadelphia_Vireo_0013_794772.jpg
CUB_200_2011/images/154.Red_eyed_Vireo/Red_Eyed_Vireo_0101_156988.jpg
CUB_200_2011/images/154.Red_eyed_Vireo/Red_Eyed_Vireo_0006_157025.jpg
CUB_200_2011/images/154.Red_eyed_Vireo/Red_Eyed_Vireo_0041_156954.jpg
CUB_200_2011/images/154.Red_eyed_Vireo/Red_Eyed_Vireo_0115_157004.jpg
CUB_200_2011/images/154.Red_eyed_Vireo/Red_Eyed_Vireo_0056_156968.jpg
CUB_200_2011/images/155.Warbling_Vireo/Warbling_Vireo_0075_158480.jpg
CUB_200_2011/images/155.Warbling_Vireo/Warbling_Vireo_0061_158494.jpg
CUB_200_2011/images/155.Warbling_Vireo/Warbling_Vireo_0004_158376.jpg
CUB_200_2011/images/155.Warbling_Vireo/Warbling_Vireo_0030_158488.jpg
CUB_200_2011/images/155.Warbling_Vireo/Warbling_Vireo_0077_158427.jpg
CUB_200_2011/images/156.White_eyed_Vireo/White_Eyed_Vireo_0042_159012.jpg
CUB_200_2011/images/156.White_eyed_Vireo/White_Eyed_Vireo_0033_159079.jpg
CUB_200_2011/images/156.White_eyed_Vireo/White_Eyed_Vireo_0126_159341.jpg
CUB_200_2011/images/156.White_eyed_Vireo/White_Eyed_Vireo_0071_159072.jpg
CUB_200_2011/images/156.White_eyed_Vireo/White_Eyed_Vireo_0016_158978.jpg
CUB_200_2011/images/157.Yellow_throated_Vireo/Yellow_Throated_Vireo_0066_795007.jpg
CUB_200_2011/images/157.Yellow_throated_Vireo/Yellow_Throated_Vireo_0032_159632.jpg
CUB_200_2011/images/157.Yellow_throated_Vireo/Yellow_Throated_Vireo_0017_794988.jpg
CUB_200_2011/images/157.Yellow_throated_Vireo/Yellow_Throated_Vireo_0025_795009.jpg
CUB_200_2011/images/157.Yellow_throated_Vireo/Yellow_Throated_Vireo_0058_794994.jpg
CUB_200_2011/images/158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0073_797138.jpg
CUB_200_2011/images/158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0081_159963.jpg
CUB_200_2011/images/158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0071_797108.jpg
CUB_200_2011/images/158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0105_797143.jpg
CUB_200_2011/images/158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0052_797125.jpg
CUB_200_2011/images/159.Black_and_white_Warbler/Black_And_White_Warbler_0057_160037.jpg
CUB_200_2011/images/159.Black_and_white_Warbler/Black_And_White_Warbler_0035_160102.jpg
CUB_200_2011/images/159.Black_and_white_Warbler/Black_And_White_Warbler_0119_160898.jpg
CUB_200_2011/images/159.Black_and_white_Warbler/Black_And_White_Warbler_0102_160073.jpg
CUB_200_2011/images/159.Black_and_white_Warbler/Black_And_White_Warbler_0022_160512.jpg
CUB_200_2011/images/160.Black_throated_Blue_Warbler/Black_Throated_Blue_Warbler_0050_161154.jpg
CUB_200_2011/images/160.Black_throated_Blue_Warbler/Black_Throated_Blue_Warbler_0130_161682.jpg
CUB_200_2011/images/160.Black_throated_Blue_Warbler/Black_Throated_Blue_Warbler_0133_161539.jpg
CUB_200_2011/images/160.Black_throated_Blue_Warbler/Black_Throated_Blue_Warbler_0054_161158.jpg
CUB_200_2011/images/160.Black_throated_Blue_Warbler/Black_Throated_Blue_Warbler_0024_161619.jpg
CUB_200_2011/images/161.Blue_winged_Warbler/Blue_Winged_Warbler_0071_161900.jpg
CUB_200_2011/images/161.Blue_winged_Warbler/Blue_Winged_Warbler_0035_161741.jpg
CUB_200_2011/images/161.Blue_winged_Warbler/Blue_Winged_Warbler_0054_161862.jpg
CUB_200_2011/images/161.Blue_winged_Warbler/Blue_Winged_Warbler_0023_161774.jpg
CUB_200_2011/images/161.Blue_winged_Warbler/Blue_Winged_Warbler_0040_161883.jpg
CUB_200_2011/images/162.Canada_Warbler/Canada_Warbler_0113_162403.jpg
CUB_200_2011/images/162.Canada_Warbler/Canada_Warbler_0064_162417.jpg
CUB_200_2011/images/162.Canada_Warbler/Canada_Warbler_0091_162378.jpg
CUB_200_2011/images/162.Canada_Warbler/Canada_Warbler_0016_162411.jpg
CUB_200_2011/images/162.Canada_Warbler/Canada_Warbler_0080_162392.jpg
CUB_200_2011/images/163.Cape_May_Warbler/Cape_May_Warbler_0012_162701.jpg
CUB_200_2011/images/163.Cape_May_Warbler/Cape_May_Warbler_0103_162972.jpg
CUB_200_2011/images/163.Cape_May_Warbler/Cape_May_Warbler_0022_162912.jpg
CUB_200_2011/images/163.Cape_May_Warbler/Cape_May_Warbler_0005_163197.jpg
CUB_200_2011/images/163.Cape_May_Warbler/Cape_May_Warbler_0032_162659.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0039_163420.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0020_163353.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0014_797226.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0072_163200.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0080_163399.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0128_163696.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0097_163750.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0094_164152.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0105_163996.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0101_164324.jpg
CUB_200_2011/images/166.Golden_winged_Warbler/Golden_Winged_Warbler_0079_794820.jpg
CUB_200_2011/images/166.Golden_winged_Warbler/Golden_Winged_Warbler_0046_794828.jpg
CUB_200_2011/images/166.Golden_winged_Warbler/Golden_Winged_Warbler_0061_164516.jpg
CUB_200_2011/images/166.Golden_winged_Warbler/Golden_Winged_Warbler_0068_794825.jpg
CUB_200_2011/images/166.Golden_winged_Warbler/Golden_Winged_Warbler_0011_794812.jpg
CUB_200_2011/images/167.Hooded_Warbler/Hooded_Warbler_0040_165173.jpg
CUB_200_2011/images/167.Hooded_Warbler/Hooded_Warbler_0001_164704.jpg
CUB_200_2011/images/167.Hooded_Warbler/Hooded_Warbler_0021_165057.jpg
CUB_200_2011/images/167.Hooded_Warbler/Hooded_Warbler_0058_164674.jpg
CUB_200_2011/images/167.Hooded_Warbler/Hooded_Warbler_0053_164631.jpg
CUB_200_2011/images/168.Kentucky_Warbler/Kentucky_Warbler_0008_165369.jpg
CUB_200_2011/images/168.Kentucky_Warbler/Kentucky_Warbler_0035_795878.jpg
CUB_200_2011/images/168.Kentucky_Warbler/Kentucky_Warbler_0050_165278.jpg
CUB_200_2011/images/168.Kentucky_Warbler/Kentucky_Warbler_0071_165342.jpg
CUB_200_2011/images/168.Kentucky_Warbler/Kentucky_Warbler_0072_165305.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0041_165709.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0092_165807.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0029_165567.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0030_165782.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0053_165682.jpg
CUB_200_2011/images/170.Mourning_Warbler/Mourning_Warbler_0069_166559.jpg
CUB_200_2011/images/170.Mourning_Warbler/Mourning_Warbler_0035_166586.jpg
CUB_200_2011/images/170.Mourning_Warbler/Mourning_Warbler_0002_166520.jpg
CUB_200_2011/images/170.Mourning_Warbler/Mourning_Warbler_0079_166564.jpg
CUB_200_2011/images/170.Mourning_Warbler/Mourning_Warbler_0015_166535.jpg
CUB_200_2011/images/171.Myrtle_Warbler/Myrtle_Warbler_0023_166764.jpg
CUB_200_2011/images/171.Myrtle_Warbler/Myrtle_Warbler_0050_166820.jpg
CUB_200_2011/images/171.Myrtle_Warbler/Myrtle_Warbler_0043_166708.jpg
CUB_200_2011/images/171.Myrtle_Warbler/Myrtle_Warbler_0098_166794.jpg
CUB_200_2011/images/171.Myrtle_Warbler/Myrtle_Warbler_0015_166713.jpg
CUB_200_2011/images/172.Nashville_Warbler/Nashville_Warbler_0108_167259.jpg
CUB_200_2011/images/172.Nashville_Warbler/Nashville_Warbler_0098_167293.jpg
CUB_200_2011/images/172.Nashville_Warbler/Nashville_Warbler_0104_167096.jpg
CUB_200_2011/images/172.Nashville_Warbler/Nashville_Warbler_0110_167268.jpg
CUB_200_2011/images/172.Nashville_Warbler/Nashville_Warbler_0081_167234.jpg
CUB_200_2011/images/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0062_168119.jpg
CUB_200_2011/images/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0050_168166.jpg
CUB_200_2011/images/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0055_168600.jpg
CUB_200_2011/images/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0118_167640.jpg
CUB_200_2011/images/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0067_167588.jpg
CUB_200_2011/images/174.Palm_Warbler/Palm_Warbler_0083_170281.jpg
CUB_200_2011/images/174.Palm_Warbler/Palm_Warbler_0012_170857.jpg
CUB_200_2011/images/174.Palm_Warbler/Palm_Warbler_0015_169626.jpg
CUB_200_2011/images/174.Palm_Warbler/Palm_Warbler_0126_170311.jpg
CUB_200_2011/images/174.Palm_Warbler/Palm_Warbler_0136_170276.jpg
CUB_200_2011/images/175.Pine_Warbler/Pine_Warbler_0017_171678.jpg
CUB_200_2011/images/175.Pine_Warbler/Pine_Warbler_0127_171742.jpg
CUB_200_2011/images/175.Pine_Warbler/Pine_Warbler_0060_171635.jpg
CUB_200_2011/images/175.Pine_Warbler/Pine_Warbler_0056_172064.jpg
CUB_200_2011/images/175.Pine_Warbler/Pine_Warbler_0102_171147.jpg
CUB_200_2011/images/176.Prairie_Warbler/Prairie_Warbler_0073_172771.jpg
CUB_200_2011/images/176.Prairie_Warbler/Prairie_Warbler_0120_173097.jpg
CUB_200_2011/images/176.Prairie_Warbler/Prairie_Warbler_0063_172682.jpg
CUB_200_2011/images/176.Prairie_Warbler/Prairie_Warbler_0053_173290.jpg
CUB_200_2011/images/176.Prairie_Warbler/Prairie_Warbler_0080_172724.jpg
CUB_200_2011/images/177.Prothonotary_Warbler/Prothonotary_Warbler_0062_174412.jpg
CUB_200_2011/images/177.Prothonotary_Warbler/Prothonotary_Warbler_0037_173418.jpg
CUB_200_2011/images/177.Prothonotary_Warbler/Prothonotary_Warbler_0076_174118.jpg
CUB_200_2011/images/177.Prothonotary_Warbler/Prothonotary_Warbler_0070_174650.jpg
CUB_200_2011/images/177.Prothonotary_Warbler/Prothonotary_Warbler_0110_173857.jpg
CUB_200_2011/images/178.Swainson_Warbler/Swainson_Warbler_0017_174685.jpg
CUB_200_2011/images/178.Swainson_Warbler/Swainson_Warbler_0039_794859.jpg
CUB_200_2011/images/178.Swainson_Warbler/Swainson_Warbler_0051_794900.jpg
CUB_200_2011/images/178.Swainson_Warbler/Swainson_Warbler_0037_174691.jpg
CUB_200_2011/images/178.Swainson_Warbler/Swainson_Warbler_0018_174715.jpg
CUB_200_2011/images/179.Tennessee_Warbler/Tennessee_Warbler_0051_175015.jpg
CUB_200_2011/images/179.Tennessee_Warbler/Tennessee_Warbler_0019_174786.jpg
CUB_200_2011/images/179.Tennessee_Warbler/Tennessee_Warbler_0023_174977.jpg
CUB_200_2011/images/179.Tennessee_Warbler/Tennessee_Warbler_0033_174772.jpg
CUB_200_2011/images/179.Tennessee_Warbler/Tennessee_Warbler_0004_174997.jpg
CUB_200_2011/images/180.Wilson_Warbler/Wilson_Warbler_0107_175320.jpg
CUB_200_2011/images/180.Wilson_Warbler/Wilson_Warbler_0065_175924.jpg
CUB_200_2011/images/180.Wilson_Warbler/Wilson_Warbler_0129_175256.jpg
CUB_200_2011/images/180.Wilson_Warbler/Wilson_Warbler_0126_175368.jpg
CUB_200_2011/images/180.Wilson_Warbler/Wilson_Warbler_0054_175285.jpg
CUB_200_2011/images/181.Worm_eating_Warbler/Worm_Eating_Warbler_0063_795553.jpg
CUB_200_2011/images/181.Worm_eating_Warbler/Worm_Eating_Warbler_0011_795566.jpg
CUB_200_2011/images/181.Worm_eating_Warbler/Worm_Eating_Warbler_0092_795524.jpg
CUB_200_2011/images/181.Worm_eating_Warbler/Worm_Eating_Warbler_0006_176037.jpg
CUB_200_2011/images/181.Worm_eating_Warbler/Worm_Eating_Warbler_0018_795546.jpg
CUB_200_2011/images/182.Yellow_Warbler/Yellow_Warbler_0083_176292.jpg
CUB_200_2011/images/182.Yellow_Warbler/Yellow_Warbler_0096_176586.jpg
CUB_200_2011/images/182.Yellow_Warbler/Yellow_Warbler_0119_176485.jpg
CUB_200_2011/images/182.Yellow_Warbler/Yellow_Warbler_0102_176821.jpg
CUB_200_2011/images/182.Yellow_Warbler/Yellow_Warbler_0049_176526.jpg
CUB_200_2011/images/183.Northern_Waterthrush/Northern_Waterthrush_0043_177070.jpg
CUB_200_2011/images/183.Northern_Waterthrush/Northern_Waterthrush_0080_177080.jpg
CUB_200_2011/images/183.Northern_Waterthrush/Northern_Waterthrush_0022_177003.jpg
CUB_200_2011/images/183.Northern_Waterthrush/Northern_Waterthrush_0050_177331.jpg
CUB_200_2011/images/183.Northern_Waterthrush/Northern_Waterthrush_0014_177305.jpg
CUB_200_2011/images/184.Louisiana_Waterthrush/Louisiana_Waterthrush_0087_795261.jpg
CUB_200_2011/images/184.Louisiana_Waterthrush/Louisiana_Waterthrush_0001_795271.jpg
CUB_200_2011/images/184.Louisiana_Waterthrush/Louisiana_Waterthrush_0034_795242.jpg
CUB_200_2011/images/184.Louisiana_Waterthrush/Louisiana_Waterthrush_0020_795265.jpg
CUB_200_2011/images/184.Louisiana_Waterthrush/Louisiana_Waterthrush_0077_795247.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0046_177864.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0042_177887.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0024_177661.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0031_796633.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0048_177821.jpg
CUB_200_2011/images/186.Cedar_Waxwing/Cedar_Waxwing_0094_178049.jpg
CUB_200_2011/images/186.Cedar_Waxwing/Cedar_Waxwing_0016_178629.jpg
CUB_200_2011/images/186.Cedar_Waxwing/Cedar_Waxwing_0125_178921.jpg
CUB_200_2011/images/186.Cedar_Waxwing/Cedar_Waxwing_0004_179215.jpg
CUB_200_2011/images/186.Cedar_Waxwing/Cedar_Waxwing_0065_179017.jpg
CUB_200_2011/images/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0018_179831.jpg
CUB_200_2011/images/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0007_179932.jpg
CUB_200_2011/images/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0024_179876.jpg
CUB_200_2011/images/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0009_179919.jpg
CUB_200_2011/images/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0012_179905.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0056_180094.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0034_180419.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0110_180521.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0079_180388.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0088_180054.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0112_180827.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0017_181131.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0125_180780.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0086_181891.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0020_182335.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0023_794701.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0033_794721.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0027_794713.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0029_794724.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0039_794736.jpg
CUB_200_2011/images/191.Red_headed_Woodpecker/Red_Headed_Woodpecker_0020_183255.jpg
CUB_200_2011/images/191.Red_headed_Woodpecker/Red_Headed_Woodpecker_0005_183414.jpg
CUB_200_2011/images/191.Red_headed_Woodpecker/Red_Headed_Woodpecker_0068_183662.jpg
CUB_200_2011/images/191.Red_headed_Woodpecker/Red_Headed_Woodpecker_0013_182721.jpg
CUB_200_2011/images/191.Red_headed_Woodpecker/Red_Headed_Woodpecker_0095_183688.jpg
CUB_200_2011/images/192.Downy_Woodpecker/Downy_Woodpecker_0040_184061.jpg
CUB_200_2011/images/192.Downy_Woodpecker/Downy_Woodpecker_0031_184120.jpg
CUB_200_2011/images/192.Downy_Woodpecker/Downy_Woodpecker_0090_183964.jpg
CUB_200_2011/images/192.Downy_Woodpecker/Downy_Woodpecker_0005_184098.jpg
CUB_200_2011/images/192.Downy_Woodpecker/Downy_Woodpecker_0136_184534.jpg
CUB_200_2011/images/193.Bewick_Wren/Bewick_Wren_0083_185190.jpg
CUB_200_2011/images/193.Bewick_Wren/Bewick_Wren_0084_184715.jpg
CUB_200_2011/images/193.Bewick_Wren/Bewick_Wren_0015_184981.jpg
CUB_200_2011/images/193.Bewick_Wren/Bewick_Wren_0110_185216.jpg
CUB_200_2011/images/193.Bewick_Wren/Bewick_Wren_0081_185080.jpg
CUB_200_2011/images/194.Cactus_Wren/Cactus_Wren_0089_186023.jpg
CUB_200_2011/images/194.Cactus_Wren/Cactus_Wren_0097_186015.jpg
CUB_200_2011/images/194.Cactus_Wren/Cactus_Wren_0025_185696.jpg
CUB_200_2011/images/194.Cactus_Wren/Cactus_Wren_0066_186028.jpg
CUB_200_2011/images/194.Cactus_Wren/Cactus_Wren_0033_186014.jpg
CUB_200_2011/images/195.Carolina_Wren/Carolina_Wren_0113_186675.jpg
CUB_200_2011/images/195.Carolina_Wren/Carolina_Wren_0099_186237.jpg
CUB_200_2011/images/195.Carolina_Wren/Carolina_Wren_0014_186525.jpg
CUB_200_2011/images/195.Carolina_Wren/Carolina_Wren_0020_186702.jpg
CUB_200_2011/images/195.Carolina_Wren/Carolina_Wren_0128_186581.jpg
CUB_200_2011/images/196.House_Wren/House_Wren_0108_187102.jpg
CUB_200_2011/images/196.House_Wren/House_Wren_0107_187230.jpg
CUB_200_2011/images/196.House_Wren/House_Wren_0035_187708.jpg
CUB_200_2011/images/196.House_Wren/House_Wren_0094_187226.jpg
CUB_200_2011/images/196.House_Wren/House_Wren_0122_187331.jpg
CUB_200_2011/images/197.Marsh_Wren/Marsh_Wren_0056_188241.jpg
CUB_200_2011/images/197.Marsh_Wren/Marsh_Wren_0141_188796.jpg
CUB_200_2011/images/197.Marsh_Wren/Marsh_Wren_0006_188126.jpg
CUB_200_2011/images/197.Marsh_Wren/Marsh_Wren_0044_188270.jpg
CUB_200_2011/images/197.Marsh_Wren/Marsh_Wren_0039_188201.jpg
CUB_200_2011/images/198.Rock_Wren/Rock_Wren_0122_189042.jpg
CUB_200_2011/images/198.Rock_Wren/Rock_Wren_0063_189121.jpg
CUB_200_2011/images/198.Rock_Wren/Rock_Wren_0069_188969.jpg
CUB_200_2011/images/198.Rock_Wren/Rock_Wren_0111_189443.jpg
CUB_200_2011/images/198.Rock_Wren/Rock_Wren_0027_189331.jpg
CUB_200_2011/images/199.Winter_Wren/Winter_Wren_0066_189637.jpg
CUB_200_2011/images/199.Winter_Wren/Winter_Wren_0030_190311.jpg
CUB_200_2011/images/199.Winter_Wren/Winter_Wren_0075_189578.jpg
CUB_200_2011/images/199.Winter_Wren/Winter_Wren_0065_189675.jpg
CUB_200_2011/images/199.Winter_Wren/Winter_Wren_0037_190123.jpg
CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0004_190606.jpg
CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0054_190398.jpg
CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0010_190572.jpg
CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0126_190407.jpg
CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0032_190592.jpg"""

In [10]:
x = x.split("\n")

In [33]:
# Need to make 10 way 5 shot classes
class2samples = {}

In [34]:
for i in x:
    class_id = int(i.split("/")[2].split(".")[0])
    if class_id not in class2samples: class2samples[class_id] = []
    class2samples[class_id].append(i)

In [35]:
import numpy as np

In [37]:
ids = np.array(list(class2samples.keys()))

In [39]:
ids = np.random.permutation(ids)

In [44]:
n_classes_split = 10
for idx, i in enumerate(ids,1):
    samples = class2samples[i]
    for sample in samples: 
        print(sample, flush=True)
    if idx % n_classes_split == 0:
        print("======", flush=True)

CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0023_796582.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0052_796599.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0054_116850.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0064_796573.jpg
CUB_200_2011/images/123.Henslow_Sparrow/Henslow_Sparrow_0070_796571.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0127_105742.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0018_26407.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0019_106132.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0011_104921.jpg
CUB_200_2011/images/111.Loggerhead_Shrike/Loggerhead_Shrike_0033_105686.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0079_125579.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0080_125606.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparrow_0084_125532.jpg
CUB_200_2011/images/131.Vesper_Sparrow/Vesper_Sparr

CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0039_163420.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0020_163353.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0014_797226.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0072_163200.jpg
CUB_200_2011/images/164.Cerulean_Warbler/Cerulean_Warbler_0080_163399.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0023_794701.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0033_794721.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0027_794713.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0029_794724.jpg
CUB_200_2011/images/190.Red_cockaded_Woodpecker/Red_Cockaded_Woodpecker_0039_794736.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0046_177864.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/Bohemian_Waxwing_0042_177887.jpg
CUB_200_2011/images/185.Bohemian_Waxwing/B

CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0112_180827.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0017_181131.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0125_180780.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0086_181891.jpg
CUB_200_2011/images/189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0020_182335.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0012_797473.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0007_797481.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0020_797461.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0053_797478.jpg
CUB_200_2011/images/151.Black_capped_Vireo/Black_Capped_Vireo_0003_797467.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0106_104216.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0086_104755.jpg
CUB_200_2011/images/110.Geococcyx/Geococcyx_0124_104141.jpg


CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0030_165782.jpg
CUB_200_2011/images/169.Magnolia_Warbler/Magnolia_Warbler_0053_165682.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0056_180094.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0034_180419.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0110_180521.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0079_180388.jpg
CUB_200_2011/images/188.Pileated_Woodpecker/Pileated_Woodpecker_0088_180054.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0027_151456.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0077_152255.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0125_151399.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0045_151227.jpg
CUB_200_2011/images/146.Forsters_Tern/Forsters_Tern_0119_152709.jpg
CUB_200_2011/images/109.American_Redstart/American_Redstart_0036_103231.jpg
CUB_200_2011/images/109.American_Red

CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0097_163750.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0094_164152.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0105_163996.jpg
CUB_200_2011/images/165.Chestnut_sided_Warbler/Chestnut_Sided_Warbler_0101_164324.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0092_153361.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0020_153458.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0060_153190.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0119_153950.jpg
CUB_200_2011/images/147.Least_Tern/Least_Tern_0037_153637.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0068_126156.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0100_126267.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0072_127080.jpg
CUB_200_2011/images/132.White_crowned_Sparrow/White_Crowned_Sparrow_0033_127728.jpg
CUB_200_20